# 只做ESM sampling

## ESM introduction


In [4]:
import math, torch, torch.nn as nn
import matplotlib.pyplot as plt
torch.manual_seed(0)
device=torch.device("cpu")
lr=2e-3

In [3]:
def sample_data(n):
    n1=n//2
    n2=n-n1
    x1=torch.randn(n1)*1.0+(-2.0)
    x2=torch.randn(n2)*0.5+3.0

    return torch.cat([x1, x2], dim=0).unsqueeze(-1)

def logp_x(x):
    logp1 = -0.5*((x+2.0)/1.0)**2 - math.log(math.sqrt(2*math.pi)*1.0)
    logp2 = -0.5*((x-3.0)/0.5)**2 - math.log(math.sqrt(2*math.pi)*0.5)
    m = torch.maximum(logp1, logp2)

    return m+torch.log(0.5*torch.exp(logp1-m)+0.5*torch.exp(logp2-m))

def true_score(x):
    x =x.clone().requires_grad_(True)
    lp = logp_x(x)
    (grad,)= torch.autograd.grad(lp.sum(), x)

    return grad

In [ ]:
class ScoreMLP(nn.Module):
    def __init__(self, hidden=64):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(1, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, hidden), nn.SiLU(),
            nn.Linear(hidden, 1)
        )
        
    def forward(self,x):
        return self.net(x)

model=ScoreMLP().to(device)
opt=torch.optim.Adam(model.parameters(), lr)
